In [3]:
import pandas as pd
import numpy as np

### 1. 데이터 로드

In [4]:
selloutData = pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")

In [5]:
selloutData.head(1)

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0


In [6]:
selloutData["NEW_QTY"] = np.where(selloutData["QTY"] < 0, 0, selloutData["QTY"])

In [7]:
### 53주차 제거

In [8]:
selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK      int64
QTY         float64
NEW_QTY     float64
dtype: object

In [9]:
selloutData["WEEK"] = selloutData["YEARWEEK"]%100

In [10]:
def customFunction(inputColumn):

    #inputColumn = 201601
    yearValue = inputColumn/100
    yearValue = int(yearValue)
    return yearValue

In [11]:
selloutData["YEAR"] = selloutData["YEARWEEK"].apply(customFunction)

In [12]:
selloutData.head(1)

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
0,A60,PRODUCT4,201402,71.0,71.0,2,2014


In [13]:
refinedData = selloutData[selloutData.WEEK < 53]

In [14]:
refinedData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
0,A60,PRODUCT4,201402,71.0,71.0,2,2014
1,A60,PRODUCT59,201402,22275.0,22275.0,2,2014
2,A60,PRODUCT34,201402,4463.0,4463.0,2,2014
3,A60,PRODUCT47,201402,0.0,0.0,2,2014
4,A60,PRODUCT56,201402,23.0,23.0,2,2014


In [15]:
sortKey = ["REGIONID","PRODUCT","YEARWEEK"]

In [16]:
sortedData = refinedData.sort_values(sortKey)

In [17]:
sortedData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
298,A00,PRODUCT34,201401,661.0,661.0,1,2014
1757,A00,PRODUCT34,201402,679.0,679.0,2,2014
3125,A00,PRODUCT34,201403,578.0,578.0,3,2014
205,A00,PRODUCT34,201404,532.0,532.0,4,2014
4369,A00,PRODUCT34,201405,516.0,516.0,5,2014


In [18]:
step1Data = sortedData.reset_index(drop=True)

In [19]:
step1Data.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
0,A00,PRODUCT34,201401,661.0,661.0,1,2014
1,A00,PRODUCT34,201402,679.0,679.0,2,2014
2,A00,PRODUCT34,201403,578.0,578.0,3,2014
3,A00,PRODUCT34,201404,532.0,532.0,4,2014
4,A00,PRODUCT34,201405,516.0,516.0,5,2014


### 1. 기본 이동평균 함수

In [20]:
step1Data["MA"] = step1Data.NEW_QTY.rolling(
    window=15, center=True, min_periods=1).mean()

In [21]:
groupKey = ["REGIONID","PRODUCT","YEAR"]

In [22]:
groupData = step1Data.groupby(groupKey)[["NEW_QTY"]].mean()

In [23]:
groupDataResult = groupData.reset_index()

In [24]:
groupDataResult.head(3)

,REGIONID,PRODUCT,YEAR,NEW_QTY
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615
2,A00,PRODUCT34,2016,36.576923


In [25]:
groupDataResult.columns=["REGIONID","PRODUCT","YEAR","MEAN_QTY"]
groupDataResult.head(2)

,REGIONID,PRODUCT,YEAR,MEAN_QTY
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615


In [26]:
leftData = step1Data
leftData.head(1)

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,MA
0,A00,PRODUCT34,201401,661.0,661.0,1,2014,520.0


## 데이터 합치기 (속성)

In [27]:
joinKey = ["REGIONID","PRODUCT","YEAR"]

In [28]:
mergedData1 = pd.merge(leftData, groupDataResult, how="left", on=joinKey)

In [29]:
mergedData1

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,MA,MEAN_QTY
0,A00,PRODUCT34,201401,661.0,661.0,1,2014,520.000000,275.961538
1,A00,PRODUCT34,201402,679.0,679.0,2,2014,514.444444,275.961538
2,A00,PRODUCT34,201403,578.0,578.0,3,2014,516.800000,275.961538
3,A00,PRODUCT34,201404,532.0,532.0,4,2014,503.363636,275.961538
4,A00,PRODUCT34,201405,516.0,516.0,5,2014,490.750000,275.961538
5,A00,PRODUCT34,201406,423.0,423.0,6,2014,480.230769,275.961538
6,A00,PRODUCT34,201407,407.0,407.0,7,2014,468.214286,275.961538
7,A00,PRODUCT34,201408,364.0,364.0,8,2014,456.666667,275.961538
8,A00,PRODUCT34,201409,470.0,470.0,9,2014,431.533333,275.961538
9,A00,PRODUCT34,201410,538.0,538.0,10,2014,402.066667,275.961538


In [29]:
len(leftData)

123864

In [30]:
len(mergedData1)

123864

In [31]:
mergedData1.to_csv("./mergedData2.csv")

In [32]:
groupKey = ["REGIONID","PRODUCT"]

### 그룹바이함수 만들기

In [33]:
#groupDataSet = mergedData1.groupby(groupKey)

In [34]:
def groupRolling(oneGroup):

    #len(list(groupDataSet.groups))

    #oneGroup = groupDataSet.get_group(list(groupDataSet.groups)[2])
    #oneGroup.head(1)

    indexGroupData = oneGroup.reset_index(drop=True)
    indexGroupData["MA2"] = indexGroupData["NEW_QTY"].rolling(window=5,center=True,min_periods=1).mean()

    return indexGroupData

In [35]:
finalResult = mergedData1.groupby(groupKey).apply(groupRolling)

In [36]:
finalResult2 = finalResult.reset_index(drop=True)

In [37]:
finalResult2.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,MA,MEAN_QTY,MA2
0,A00,PRODUCT34,201401,661.0,661.0,1,2014,520.000000,275.961538,639.333333
1,A00,PRODUCT34,201402,679.0,679.0,2,2014,514.444444,275.961538,612.500000
2,A00,PRODUCT34,201403,578.0,578.0,3,2014,516.800000,275.961538,593.200000
3,A00,PRODUCT34,201404,532.0,532.0,4,2014,503.363636,275.961538,545.600000
4,A00,PRODUCT34,201405,516.0,516.0,5,2014,490.750000,275.961538,491.200000


regionid가 A01인 데이터를 fullData에 

In [38]:
fullData = pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")

In [39]:
a01Data =fullData[fullData.REGIONID=="A01"]
len(a01Data)

3611

In [43]:
a20Data =fullData[fullData.REGIONID=="A20"]
len(a20Data)

471

In [44]:
resultConcat = pd.concat([a01Data,a20Data],axis=0)

In [45]:
len(resultConcat)

4082